<a href="https://colab.research.google.com/github/SamytechInc/CSFIntershipAssessment2024/blob/main/Carrot_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Processing video pipeline
This is an implementation of how I would adapt a model to record (in a .json file) carrots. It records the confidence of the model, the moment it detected it and the number of different carrots (using a feature matching algorithm). It uses a webcam and YOLOv5s as placeholders of a grocery camera and a carrot-detecting model, respectively.



In [ ]:
!pip install torch
!pip install cv2
!pip install datetime
!pip install pypi-json

In [1]:
import torch
import cv2
import numpy as np
import json
from datetime import datetime
from collections import defaultdict

# Load YOLOv5 model from Ultralytics repository
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Initialize ORB detector for feature matching
orb = cv2.ORB_create()

# Function to process live video and count carrots
def process_live_video(camera_index=0, output_json='carrot_detections.json'):
    # Open the webcam
    cap = cv2.VideoCapture(camera_index)
    if not cap.isOpened():
        print("Error opening video stream or file")
        return

    # Initialize data storage
    data = {"detections": []}
    carrot_descriptors = []

    # Loop through video frames
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Preprocess the frame for YOLO
        results = model(frame)

        # Post-process the results to extract bounding boxes, labels, and confidence scores
        labels, confidences, boxes = results.xyxyn[0][:, -1], results.xyxyn[0][:, -2], results.xyxyn[0][:, :-2]

        # Iterate through detections and filter for carrots (assuming class 0 is carrot, this will depend on your model's class mappings)
        for label, confidence, box in zip(labels, confidences, boxes):
            if int(label) == 0:  # Assuming class 0 is carrot
                # Extract bounding box coordinates
                x1, y1, x2, y2 = int(box[0] * frame.shape[1]), int(box[1] * frame.shape[0]), int(box[2] * frame.shape[1]), int(box[3] * frame.shape[0])

                # Crop the detected carrot region
                carrot_region = frame[y1:y2, x1:x2]

                # Detect and compute ORB descriptors
                keypoints, descriptors = orb.detectAndCompute(carrot_region, None)

                # Feature matching to count unique carrots
                matches = []
                if descriptors is not None:
                    for existing_descriptors in carrot_descriptors:
                        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
                        matches = bf.match(descriptors, existing_descriptors)
                        if len(matches) > 10:  # Threshold to consider the same carrot
                            break
                    if len(matches) <= 10:
                        carrot_descriptors.append(descriptors)

                # Draw bounding box and label on the frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"Carrot: {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

                # Print detection info
                print(f"Carrot detected with a confidence of {confidence:.2f} at {timestamp}.")

                # Record detection in JSON data
                timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                detection = {
                    "timestamp": timestamp,
                    "confidence": float(confidence),
                    "bounding_box": [x1, y1, x2, y2]
                }
                data["detections"].append(detection)

        # Display the frame with detections
        cv2.imshow('Live Video', frame)

        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Record the count of unique carrots
    data["unique_carrots_count"] = len(carrot_descriptors)

    # Save data to JSON file
    with open(output_json, 'w') as json_file:
        json.dump(data, json_file, indent=4)

    # Release video capture and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

# Start processing live video from the default webcam (camera index 0)
process_live_video(camera_index=0, output_json='carrot_detections.json')


KeyboardInterrupt: 